In [1]:
import sys
sys.path.append('../')

import random
import requests
import urllib.parse
from bs4 import BeautifulSoup
from objects.profile import Profile
import json
from collections import OrderedDict

from selenium import webdriver
from selenium_stealth import stealth
from selenium.webdriver.common.by import By
import time
from tqdm import tqdm
import pandas as pd

from data_sources.google_scholar import scrape_publication_details, scrape_google_scholar_profile, save_google_scholar_profile, save_publications_details

In [2]:
with open('./data/individual_profiles.json', 'r') as f:
    ind_profile = json.load(f)
    
google_scholar_profile_url = []

for profile in ind_profile:
    if profile['google_scholar'] is not None:
        google_scholar_profile_url.append([profile['full_name'], profile['google_scholar']])
len(google_scholar_profile_url)

32

# Scraping Google Scholar Profile Page

In [ ]:
for profile in google_scholar_profile_url:
    name, url = profile[0], profile[1]

    g_profile = scrape_google_scholar_profile(url)
    save_google_scholar_profile(g_profile, output_filename=f'google_scholar_{name.lower().replace(" ","_")}.json')

    if random.uniform(0,1)>0.8:
        print('Sleeping')
        time.sleep(60)

# Scraping Citation Information

In [3]:
def create_driver(debug=False):

    options = webdriver.ChromeOptions()
    if debug==False:
        options.add_argument("--headless")
    options.add_experimental_option("excludeSwitches", ["enable-automation"])
    options.add_experimental_option('useAutomationExtension', False)
    driver = webdriver.Chrome(
        options=options
    )
    stealth(driver,
            # user_agent=agent,
            languages=["en-US", "en"],
            vendor="Google Inc.",
            platform="Win32",
            webgl_vendor="Intel Inc.",
            renderer="Intel Iris OpenGL Engine",
            fix_hairline=True,
            )
    return driver


In [7]:
driver = create_driver(debug=True)

for j, profile in enumerate(google_scholar_profile_url[1:]):
    name, url = profile[0], profile[1]
    with open(f'./data/google_scholar_{name.lower().replace(" ","_")}.json', 'r') as f:
        data = json.load(f)
    print(j)
    details = {}
    try:
        for i, publication in enumerate(data['publications']):
            key = publication['title_link']
            driver.get(key)
            time.sleep(random.uniform(3,6))
            details[key] = scrape_publication_details(key, driver)
            if random.uniform(0,1)>0.75:
                time.sleep(random.uniform(5,10))
            if random.uniform(0,1)>0.9:
                driver.quit()
                driver = create_driver(debug=False)
            
            if i%10==0:
                save_publications_details(details, f'google_scholar_{name.lower().replace(" ","_")}_publications.json')
    except Exception as e:
        print(f'Stopped at {j} {name}')
        print(e)

        save_publications_details(details, f'google_scholar_{name.lower().replace(" ","_")}_publications.json')
        break
    
    
    save_publications_details(details, f'google_scholar_{name.lower().replace(" ","_")}_publications.json')
    driver.quit()
    driver = create_driver(debug=False)

    time.sleep(random.uniform(4*60, 6*60))

0
Error in scraping publication details at https://scholar.google.com/citations?view_op=view_citation&hl=en&user=8A7kHCYAAAAJ&cstart=200&pagesize=100&citation_for_view=8A7kHCYAAAAJ:e_rmSamDkqQC
1
Error in scraping publication details at https://scholar.google.com/citations?view_op=view_citation&hl=en&user=TIt4ggwAAAAJ&cstart=300&pagesize=100&citation_for_view=TIt4ggwAAAAJ:2ywjKiB__4kC
Error in scraping publication details at https://scholar.google.com/citations?view_op=view_citation&hl=en&user=TIt4ggwAAAAJ&cstart=300&pagesize=100&citation_for_view=TIt4ggwAAAAJ:HtEfBTGE9r8C
2
Error in scraping publication details at https://scholar.google.com/citations?view_op=view_citation&hl=en&user=B_ouhTgAAAAJ&cstart=100&pagesize=100&citation_for_view=B_ouhTgAAAAJ:XiSMed-E-HIC
3
Error in scraping publication details at https://scholar.google.com/citations?view_op=view_citation&hl=en&user=FJodrCcAAAAJ&cstart=300&pagesize=100&citation_for_view=FJodrCcAAAAJ:nVrZBo8bIpAC
Error in scraping publication de

KeyboardInterrupt: 

In [20]:
from striprtf.striprtf import rtf_to_text

with open("problem_google_scholar_publication.rtf") as infile:
    content = infile.read()
    text = rtf_to_text(content)
text = text.split('\n')
faulty_url = []
for t in text:
    if not t.isnumeric() and t!='':
        faulty_url.append(t.split(' at ')[1])

In [21]:
driver = create_driver(debug=True)

for j, profile in enumerate(google_scholar_profile_url[1:]):
    name, url = profile[0], profile[1]
    with open(f'./data/google_scholar_{name.lower().replace(" ","_")}.json', 'r') as f:
        data = json.load(f)
    details = {}
    print(len(faulty_url))
    for i, publication in enumerate(data['publications']):
        key = publication['title_link']
        if key in faulty_url:
            driver.get(key)
            time.sleep(random.uniform(3,6))
            details[key] = scrape_publication_details(key, driver)
            if random.uniform(0,1)>0.75:
                time.sleep(random.uniform(5,10))
            if random.uniform(0,1)>0.9:
                driver.quit()
                driver = create_driver(debug=False)
            faulty_url.remove(key)

        save_publications_details(details, f'google_scholar_{name.lower().replace(" ","_")}_publications.json')

    if len(faulty_url)==0:
        break
driver.quit()

79
78
76
75
73
72
72
72
56
56
51
37
34
31
31
31
5
5
4
2
1
0
0
0
0
0
0
0
0
0
0
